# Stream-validate LLM responses


In [16]:
# Few imports and global variables
from rich import print
import guardrails as gd
import litellm
from IPython.display import clear_output
import time

### Setup

Install the necessary validators from Guardrails hub in your CLI.

```bash
!guardrails hub install hub://guardrails/valid_range
!guardrails hub install hub://guardrails/uppercase
!guardrails hub install hub://guardrails/lowercase
!guardrails hub install hub://guardrails/one_line
```

### 1. For structured JSON output


#### Define the prompt and output schema


In [17]:
from pydantic import BaseModel, Field
from guardrails.hub import LowerCase, UpperCase, ValidRange, OneLine
from typing import List

prompt = """
Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
"""

doctors_notes = """152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.
The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been using cream for 2 weeks and also suffers from diabetes."""


class Symptom(BaseModel):
    symptom: str = Field(description="Symptom that a patient is experiencing")
    affected_area: str = Field(
        description="What part of the body the symptom is affecting",
        validators=[
            LowerCase(on_fail="fix"),
        ],
    )


class Medication(BaseModel):
    medication: str = Field(
        description="Name of the medication the patient is taking",
        validators=[UpperCase(on_fail="fix")],
    )
    response: str = Field(description="How the patient is responding to the medication")


class PatientInfo(BaseModel):
    gender: str = Field(description="Patient's gender")
    age: int = Field(
        description="Patient's age",
        validators=[ValidRange(min=0, max=100, on_fail="fix")],
    )
    symptoms: List[Symptom] = Field(
        description="Symptoms that the patient is currently experiencing. Each symptom should be classified into  separate item in the list."
    )
    current_meds: List[Medication] = Field(
        description="Medications the patient is currently taking and their response"
    )
    miscellaneous: str = Field(
        description="Any other information that is relevant to the patient's health; something that doesn't fit into the other categories.",
        validators=[LowerCase(on_fail="fix"), OneLine(on_fail="fix")],
    )

#### Create the Guard object


In [18]:
guard = gd.Guard.from_pydantic(output_class=PatientInfo, prompt=prompt)

##### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [19]:
# Wrap the litellm OpenAI API call with the `guard` object
raw_llm_output, validated_output, *rest = guard(
    litellm.completion,
    model="gpt-3.5-turbo",
    prompt_params={"doctors_notes": doctors_notes},
    max_tokens=1024,
    temperature=0.3,
)

# Print the validated output from the LLM
print(validated_output)

/Users/wyatt/Projects/guardrails/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{
    'gender': 'female',
    'age': 100,
    'symptoms': [
        {'symptom': 'chronic macular rash', 'affected_area': 'face and hair'},
        {'symptom': 'itchy, flaky and slightly scaly rash', 'affected_area': 'beard, eyebrows and nares'}
    ],
    'current_meds': [{'medication': 'OTC STEROID CREAM', 'response': 'moderate'}],
    'miscellaneous': 'patient also suffers from diabetes'
}

In [20]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
    │ │ patient's information.                                                                                  │ │
    │ │                                                                                                         │ │
    │ │ 152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.          │ │
    │ │ The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been   │ │
    │ │ using cream for 2 weeks and also suffers from diabetes.                                                 │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is a JSON Schema that describes the information to extract from this document and the tags  │ │
    │ │ to extract it into.                                                                                     │ │
    │ │                                                                                                         │ │
    │ │ {"$defs": {"Medication": {"properties": {"medication": {"description": "Name of the medication the      │ │
    │ │ patient is taking", "title": "Medication", "type": "string", "validators": [{"rail_alias":              │ │
    │ │ "guardrails/uppercase"}]}, "response": {"description": "How the patient is responding to the            │ │
    │ │ medication", "title": "Response", "type": "string"}}, "required": ["medication", "response"], "title":  │ │
    │ │ "Medication", "type": "object"}, "Symptom": {"properties": {"symptom": {"description": "Symptom that a  │ │
    │ │ patient is experiencing", "title": "Symptom", "type": "string"}, "affected_area": {"description": "What │ │
    │ │ part of the body the symptom is affecting", "title": "Affected Area", "type": "string", "validators":   │ │
    │ │ [{"rail_alias": "guardrails/lowercase"}]}}, "required": ["symptom", "affected_area"], "title":          │ │
    │ │ "Symptom", "type": "object"}}, "properties": {"gender": {"description": "Patient's gender", "title":    │ │
    │ │ "Gender", "type": "string"}, "age": {"description": "Patient's age", "title": "Age", "type": "integer", │ │
    │ │ "validators": [{"rail_alias": "guardrails/valid_range"}]}, "symptoms": {"description": "Symptoms that   │ │
    │ │ the patient is currently experiencing. Each symptom should be classified into  separate item in the     │ │
    │ │ list.", "items": {"$ref": "#/$defs/Symptom"}, "title": "Symptoms", "type": "array"}, "current_meds":    │ │
    │ │ {"description": "Medications the patient is currently taking and their response", "items": {"$ref":     │ │
    │ │ "#/$defs/Medication"}, "title": "Current Meds", "type": "array"}, "miscellaneous": {"description": "Any │ │
    │ │ other information that is relevant to the patient's health; something that doesn't fit into the other   │ │
    │ │ categories.", "title": "Miscellaneous", "type": "string", "validators": [{"rail_alias":                 │ │
    │ │ "guardrails/lowercase"}, {"rail_alias": "guardrails/one_line"}]}}, "required": ["gender", "age",        │ │
    │ │ "symptoms", "current_meds", "miscellaneous"], "type": "object", "title": "PatientInfo"}                 │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other t

##### Example 2: Streaming

Set the `stream` parameter to `True`


In [21]:
# Wrap the litellm OpenAI API call with the `guard` object
fragment_generator = guard(
    litellm.completion,
    model="gpt-3.5-turbo",
    prompt_params={"doctors_notes": doctors_notes},
    max_tokens=1024,
    temperature=0,
    stream=True,
)


for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.5)

ValidationOutcome(
    raw_llm_output='{\n  "gender": "female",\n  "age": 152,\n  "symptoms": [\n    {\n      "symptom": "chronic 
macular rash",\n      "affected_area": "face and hair"\n    }\n  ],\n  "current_meds": [\n    {\n      
"medication": "OTC steroid cream",\n      "response": "Moderate"\n    }\n  ],\n  "miscellaneous": "patient also 
suffers from diabetes"\n}',
    validated_output={
        'gender': 'female',
        'age': 100,
        'symptoms': [{'symptom': 'chronic macular rash', 'affected_area': 'face and hair'}],
        'current_meds': [{'medication': 'OTC STEROID CREAM', 'response': 'Moderate'}],
        'miscellaneous': 'patient also suffers from diabetes'
    },
    reask=None,
    validation_passed=True,
    error=None
)

In [22]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
    │ │ patient's information.                                                                                  │ │
    │ │                                                                                                         │ │
    │ │ 152 y/o female with chronic macular rash to face and hair, worse in beard, eyebrows and nares.          │ │
    │ │ The rash is itchy, flaky and slightly scaly. Moderate response to OTC steroid cream. Patient has been   │ │
    │ │ using cream for 2 weeks and also suffers from diabetes.                                                 │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is a JSON Schema that describes the information to extract from this document and the tags  │ │
    │ │ to extract it into.                                                                                     │ │
    │ │                                                                                                         │ │
    │ │ {"$defs": {"Medication": {"properties": {"medication": {"description": "Name of the medication the      │ │
    │ │ patient is taking", "title": "Medication", "type": "string", "validators": [{"rail_alias":              │ │
    │ │ "guardrails/uppercase"}]}, "response": {"description": "How the patient is responding to the            │ │
    │ │ medication", "title": "Response", "type": "string"}}, "required": ["medication", "response"], "title":  │ │
    │ │ "Medication", "type": "object"}, "Symptom": {"properties": {"symptom": {"description": "Symptom that a  │ │
    │ │ patient is experiencing", "title": "Symptom", "type": "string"}, "affected_area": {"description": "What │ │
    │ │ part of the body the symptom is affecting", "title": "Affected Area", "type": "string", "validators":   │ │
    │ │ [{"rail_alias": "guardrails/lowercase"}]}}, "required": ["symptom", "affected_area"], "title":          │ │
    │ │ "Symptom", "type": "object"}}, "properties": {"gender": {"description": "Patient's gender", "title":    │ │
    │ │ "Gender", "type": "string"}, "age": {"description": "Patient's age", "title": "Age", "type": "integer", │ │
    │ │ "validators": [{"rail_alias": "guardrails/valid_range"}]}, "symptoms": {"description": "Symptoms that   │ │
    │ │ the patient is currently experiencing. Each symptom should be classified into  separate item in the     │ │
    │ │ list.", "items": {"$ref": "#/$defs/Symptom"}, "title": "Symptoms", "type": "array"}, "current_meds":    │ │
    │ │ {"description": "Medications the patient is currently taking and their response", "items": {"$ref":     │ │
    │ │ "#/$defs/Medication"}, "title": "Current Meds", "type": "array"}, "miscellaneous": {"description": "Any │ │
    │ │ other information that is relevant to the patient's health; something that doesn't fit into the other   │ │
    │ │ categories.", "title": "Miscellaneous", "type": "string", "validators": [{"rail_alias":                 │ │
    │ │ "guardrails/lowercase"}, {"rail_alias": "guardrails/one_line"}]}}, "required": ["gender", "age",        │ │
    │ │ "symptoms", "current_meds", "miscellaneous"], "type": "object", "title": "PatientInfo"}                 │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other t

As you can see here, the outputs in both examples match. The only difference is that, in the streaming example, the outputs are returned as soon as they are received and validated by Guardrails. In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. In other words, when streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.


### 2. For unstructured text output


#### Define the prompt and Guard object with validators


In [23]:
from guardrails.hub import UpperCase, OneLine

prompt = """
Generate a short description of large language models. Each new sentence should be on another line.
"""

guard = gd.Guard.from_string(
    validators=[
        UpperCase(on_fail="fix"),
        OneLine(on_fail="fix"),
    ],
    description="testmeout",
    prompt=prompt,
)

#### Example 1: No streaming

By default, the `stream` parameter is set to `False`


In [24]:
# Wrap the litellm OpenAI API call with the `guard` object
raw, validated, *rest = guard(
    litellm.completion,
    model="gpt-3.5-turbo",
    max_tokens=50,
    temperature=0.1,
)

# Print the raw and validated outputs
print(f"Raw output:\n{raw}")
print(f"Validated output:\n{validated}")

Raw output:
Large language models are advanced artificial intelligence systems that are trained on vast amounts of text data. 
They are capable of generating human-like text and understanding natural language. 
These models have the ability to perform a wide range of language-related tasks, such as

Validated output:
LARGE LANGUAGE MODELS ARE ADVANCED ARTIFICIAL INTELLIGENCE SYSTEMS THAT ARE TRAINED ON VAST AMOUNTS OF TEXT DATA.

#### Example 2: With streaming

Set the `stream` parameter to `True`


In [25]:
# Wrap the litellm OpenAI API call with the `guard` object
fragment_generator = guard(
    litellm.completion,
    model="gpt-3.5-turbo",
    max_tokens=50,
    temperature=0.1,
    stream=True,
)

for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.1)

ValidationOutcome(
    raw_llm_output=' range',
    validated_output=' range',
    reask=None,
    validation_passed=False,
    error=None
)

In [26]:
# See guard history
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Generate a short description of large language models. Each new sentence should be on another line.     │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ Large language models are advanced artificial intelligence systems that are trained on vast amounts of  │ │
    │ │ text data.                                                                                              │ │
    │ │ They are capable of generating human-like text and understanding context in a way that was previously   │ │
    │ │ thought impossible.                                                                                     │ │
    │ │ These models have the ability to perform a wide range                                                   │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ None                                                                                                    │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the outputs in both examples match. The only difference is that, in the streaming example, the outputs are returned as soon as they are received and validated by Guardrails. In the non-streaming example, the outputs are returned only after the entire request has been processed by the API. In other words, when streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.